# Web Scraper to make Image datasets

This is a small script to scrape images from google searching for a particular term, and have images of each term in a separate folder. This is extremely useful if you want to do deep learning with images, and need images from google to make a dataset.

Idea and starter Credits : https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d



## Things to do before this script works
1. Install selenium (with conda you can use "pip install selenium") 
2. Download google chrome, if downloaded find chrome version. 
3. from https://chromedriver.chromium.org/downloads, download the correct chromedriver version. (you can use other browsers too, but have to improvise)



### Variables to adjust

1. set DRIVER_PATH to the path of chromedriver
2. set search_term to an array of strings for which you want images
3. set number_images to the number of images you want for each class
4. set target_path to the path where you want images dataset created.

In [4]:
DRIVER_PATH = "D:\Deep Learning\Scraping\chromedriver.exe"
search_term = ["Violin","Guitar","Cello"]
number_images = 100

target_path = '../images'

In [5]:
from selenium import webdriver
import os 
import time
import requests
import io
from PIL import Image
import hashlib

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
    


def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSGeeNIMp5wwUCfL5_VnnSvojAda19tQJhT3w&usqp=CAU - as ../images\violin\4119e2c266.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQbQkrEKWg0R8g7sTiiidThafa60fmxat5AtA&usqp=CAU - as ../images\violin\787d783eb5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ4E8ZWE938UW7FtAvIdjJVSHITUiHqt2P0gQ&usqp=CAU - as ../images\violin\aff6da71f3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS53XKW_rMDY4negtCc4ElPtLBt7lsZxBQSOQ&usqp=CAU - as ../images\violin\09f909b570.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSlRQwfp8F_Z27llc4u3as4pk-O3oRiQ0QQfg&usqp=CAU - as ../images\violin\43a7f8ae74.jpg
SUCCESS - saved https://thumbs.static-thomann.de/thumb/orig/pics/bdb/172518/14389374_800.jpg - as ../images\violin\505f6722b6.jpg
SUCCESS - 

SUCCESS - saved https://cdn.britannica.com/27/176327-050-67702AB2/Violin.jpg - as ../images\violin\a4fb075755.jpg
SUCCESS - saved https://cdn.shopify.com/s/files/1/2994/5334/products/Bunnel_Pupil-Outfit_1200x1200.jpg?v=1591913547 - as ../images\violin\0039bbd0e5.jpg
SUCCESS - saved https://www.connollymusic.com/hs-fs/hubfs/poopular-music-violin-blog.jpg?width=760&name=poopular-music-violin-blog.jpg - as ../images\violin\5c79d4d3a5.jpg
SUCCESS - saved https://www.nationalww2museum.org/sites/default/files/styles/wide_large/public/2017-07/stalag-symphony-violin-2012.391.001.4000x4000.jpg - as ../images\violin\af75fc07de.jpg
SUCCESS - saved https://www.romania-insider.com/sites/default/files/styles/article_large_image/public/2019-12/violin_hora_facebook_page.jpg - as ../images\violin\6cccb30788.jpg
SUCCESS - saved https://imgaz1.staticbg.com/thumb/large/oaupload/banggood/images/3E/84/4afb315a-7947-4cd8-963f-aa9688c998dc.jpg - as ../images\violin\b03eb7db53.jpg
SUCCESS - saved https://encry

SUCCESS - saved https://www.taylorswift.com/sites/g/files/aaj11631/f/guitar_updatedmock.png - as ../images\guitar\2138a9398a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRF7f2_bKanXl-_DDfmxF_8mcNxexNXtyqu6Q&usqp=CAU - as ../images\guitar\074773ba04.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSVS5PBBCvI8-6_CxjVnoa8hRDBWRjNucyr8w&usqp=CAU - as ../images\guitar\ddb974affe.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTWvhGkfnp3ABu2oV3uKeiYOCuIsE_1EcLeFA&usqp=CAU - as ../images\guitar\802afddb78.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRpT1aAwHt7nUnJA9VMnBPRxXRfbVFALwOL1w&usqp=CAU - as ../images\guitar\9f48baec04.jpg
SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/41jIw1mUV4L._AC_.jpg - as ../images\guitar\6c3031a481.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSWBxuaCVgS3lBm8ek0dDopbdsuCqn5-bqmGg&usqp=CAU - as ../im